# Using Qiskit to Simulate DUCC Hamiltonians with VQE


This is a working notebook to play around with routines to get these Hamiltonian simulations on a quantum computer.

08/2019 - Right now I am just copying my code for a single hamiltonian. I am then going to run pytket with it.

In [1]:
import numpy as np
import yaml as yml
from yaml import SafeLoader as Loader
import Load_Hamiltonians as lh
import os

# This is needed for the import
from qiskit.chemistry import QMolecule as qm
from qiskit import Aer

In [9]:
# IBM backend
backend1 = Aer.get_backend('statevector_simulator')
backend2 = Aer.get_backend('qasm_simulator')

#Importing data generated from NW Chem to run experiments
#Li2_cc-pVTZ_4_ORBITALS_Li2-4_ORBITALS-ccpVTZ-2_1384
root_dir = '/Users/mmetcalf/Dropbox/Quantum Embedding/Codes/Lithium_Downfolding/Qiskit Chem/Hamiltonian_Downfolding_IBM/IntegralData/H2_MEKENA/'
NW_data_file = str(root_dir+'h2_ccpvtz_ccsd_0_80au_ducc_1_3.yaml')
# NW_data_file = str('H2.yaml')
OE_data_file = str(root_dir+ 'h2_ccpvtz_ccsd_0_80au.FOCK')
# NW_data_file = 'H2.yaml'
doc_nw = open(NW_data_file,'r')
doc_oe = open(OE_data_file,'r')
data = yml.load(doc_nw,Loader)
content_oe = doc_oe.readlines()

map_type = str('jordan_wigner')
truncation_threshold = 0.1

n_spatial_orbitals = data['integral_sets'][0]['n_orbitals']
nuclear_repulsion_energy = data['integral_sets'][0]['coulomb_repulsion']['value']
n_orbitals = 2*n_spatial_orbitals
n_particles = data['integral_sets'][0]['n_electrons']
dist = 2*data['integral_sets'][0]['geometry']['atoms'][1]['coords'][2]

if map_type == 'parity':
    #For two-qubit reduction
    n_qubits = n_orbitals-2
else:
    n_qubits = n_orbitals
    
#Populating the QMolecule class with the data to make calculations easier
qm.num_orbitals = n_spatial_orbitals
qm.num_alpha = n_particles//2
qm.num_beta = n_particles//2
#This is just a place holder for freezing the core that I need for the MP2 calculations
qm.core_orbitals = 0
qm.nuclear_repulsion_energy = nuclear_repulsion_energy
qm.hf_energy = data['integral_sets'][0]['scf_energy']['value']

###################Get Orbital Energies from FOCK file########################
orbital_energies = []
found = False
count = 0
for line in content_oe:
    if 'Eigenvalues:' in line.split()[0]:
        found =True
    if found and len(line.split()) > 1:
        orbital_energies.append(float(line.split()[1]))
        count += 1
        if count >= n_spatial_orbitals:
            break
qm.orbital_energies = orbital_energies

active_occ = n_particles//2
# active_virt = (n_orbitals-n_particles)//2
active_virt = 1
active_occ_list = np.arange(active_occ)
active_occ_list = active_occ_list.tolist()
#print('Orbitals {} are occupied'.format(active_occ_list))
active_virt_list = np.arange(active_virt)
active_virt_list = active_virt_list.tolist()

one_electron_import = data['integral_sets'][0]['hamiltonian']['one_electron_integrals']['values']
two_electron_import = data['integral_sets'][0]['hamiltonian']['two_electron_integrals']['values']

With everything imported from the files, the Hamiltonian can be constructed using the LoadHamiltonians file. These arrays are then used to construct a fermionic operator -> map fermionic operator -> get qubit operator.

In [10]:
import Load_Hamiltonians as lh
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import Operator

one_electron_spatial_integrals, two_electron_spatial_integrals = lh.get_spatial_integrals(one_electron_import,two_electron_import,n_spatial_orbitals)
one_electron_spatial_integrals, two_electron_spatial_integrals = lh.trunctate_spatial_integrals(one_electron_spatial_integrals,two_electron_spatial_integrals,.001)

#For the MP2 calculation
qm.mo_eri_ints = two_electron_spatial_integrals

h1, h2 = lh.convert_to_spin_index(one_electron_spatial_integrals,two_electron_spatial_integrals,n_spatial_orbitals, .001)
fop = FermionicOperator(h1, h2)
qop_paulis = fop.mapping(map_type)
qop = Operator(paulis=qop_paulis.paulis)

## Quantum Constructions

### Initial State and UCCSD objects

Now that there is a qubit operator we can use classes in qiskit chemistry to make circuits

In [12]:
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD

init_state = HartreeFock(n_qubits, n_orbitals, n_particles, map_type,two_qubit_reduction=False)
#Keep in mind Jarrod didn't use any singles for his ansatz. Maybe just stick to some local doubles, diagonal cancel
var_op = UCCSD(num_qubits=n_qubits, depth=1, num_orbitals=n_orbitals, num_particles=n_particles, active_occupied=active_occ_list,\
               active_unoccupied=active_virt_list,initial_state=init_state, qubit_mapping=map_type, mp2_reduction=True)

# I will be playing around with the UCCSD circuit
dumpy_params = np.random.rand(var_op.num_parameters)
var_cirq = var_op.construct_circuit(dumpy_params)
print(var_cirq)

ValueError: Computed num qubits 2 does not match actual 8

#### Using CQC's Pytket to optimize the variational anzats circuit

In [13]:
from pytket.qiskit import *

def print_tkcirc_via_qiskit(tkcirc):
    copy_tkcirc = tkcirc.copy()
    qiskit_qcirc = tk_to_qiskit(copy_tkcirc)
    print(qiskit_qcirc)

tk_cirq = qiskit_to_tk(var_cirq)
print('{} depth of unpotimized UCCSD circuit'.format(tk_cirq.depth()))
#print_tkcirc_via_qiskit(tk_cirq)

521 depth of unpotimized UCCSD circuit


In [19]:
from pytket import Transform

Transform.RemoveRedundancies().apply(tk_cirq)
Transform.CommuteRzRxThroughCX().apply(tk_cirq)
Transform.OptimisePhaseGadgets().apply(tk_cirq)
print('{} depth of UCCSD circuit with redundancies removed'.format(tk_cirq.depth()))
print_tkcirc_via_qiskit(tk_cirq)

296 depth of UCCSD circuit with redundancies removed
             ┌────────────────┐     ┌───┐          »
q_0: |0>─────┤ U3(1.5*pi,0,0) ├─────┤ X ├───────■──»
          ┌──┴────────────────┴─┐   └─┬─┘┌───┐┌─┴─┐»
q_1: |0>──┤ U3(0.5*pi,0,1.0*pi) ├─────┼──┤ X ├┤ X ├»
          ├─────────────────────┤     │  └─┬─┘└───┘»
q_2: |0>──┤ U3(0.5*pi,0,1.0*pi) ├─────┼────┼───────»
          ├─────────────────────┤     │    │       »
q_3: |0>──┤ U3(0.5*pi,0,1.0*pi) ├─────┼────┼───────»
          └──┬────────────────┬─┘     │    │       »
q_4: |0>─────┤ U3(1.5*pi,0,0) ├───────┼────■───────»
        ┌────┴────────────────┴────┐  │            »
q_5: |0>┤ U3(0.5*pi,1.5*pi,0.5*pi) ├──■────────────»
        └──────────────────────────┘               »
q_6: |0>───────────────────────────────────────────»
        ┌──────────────────────────┐               »
q_7: |0>┤ U3(0.5*pi,1.5*pi,0.5*pi) ├───────────────»
        └──────────────────────────┘               »
«                                       ┌───┐ 

The rest of the circuit optimization is going to come from symmetries in the Hilbert space